In [1]:
#import dependencies
import pandas as pd
import json
from bs4 import BeautifulSoup
import requests
import re
import sqlalchemy
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

In [2]:
#create function that will turn JSON encoded data into Python objects.
def load_credits(path):
    df = pd.read_csv(path)
    json_columns = ['cast']
    for column in json_columns:
        df[column] = df[column].apply(json.loads)
    return df

In [3]:
#load csv dataset
credits = load_credits('movies.csv')
credits.head(5)

,movie_id,title,cast,crew
0,19995,Avatar,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{'cast_id': 5, 'character': 'John Carter', 'c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
#find keyvalues for cast column
print(sorted(credits.cast.iloc[0][0].keys()))

['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order']


In [5]:
#create list of actor
actor_name = [actor['name'] for actor in credits['cast'].iloc[0]]

In [6]:
#function to grab lead actor in each movie
def get_lead_actor(container, index_values):
    # return a missing value rather than an error upon indexing/key failure
    result = container
    try:
        for idx in index_values:
            result = result[idx]
        return result
    except IndexError or KeyError:
        return pd.np.nan

In [7]:
credits['lead_actor'] = credits.cast.apply(lambda x: get_lead_actor(x, [0, 'name']))
credits.head(3)

,movie_id,title,cast,crew,lead_actor
0,19995,Avatar,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",Sam Worthington
1,285,Pirates of the Caribbean: At World's End,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",Johnny Depp
2,206647,Spectre,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",Daniel Craig


In [8]:
#reduce the size of dataframe to 10 movies
credits20 = credits.head(20)

In [9]:
#cleanup dataframe
movies_df = credits20.drop(['movie_id', 'cast', 'crew'], axis = 1)

In [10]:
movies_df

,title,lead_actor
0,Avatar,Sam Worthington
1,Pirates of the Caribbean: At World's End,Johnny Depp
2,Spectre,Daniel Craig
3,The Dark Knight Rises,Christian Bale
4,John Carter,Taylor Kitsch
5,Spider-Man 3,Tobey Maguire
6,Tangled,Zachary Levi
7,Avengers: Age of Ultron,Robert Downey Jr.
8,Harry Potter and the Half-Blood Prince,Daniel Radcliffe
9,Batman v Superman: Dawn of Justice,Ben Affleck


In [11]:
#function to scrape wikipedia.org for each actor, grabbing birthday, nickname(which is really full name), and birthplace
def to_scrape(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36'}
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.content, 'html.parser')
        bday = soup.find('span', {'class': 'bday'}).text
        nickname = soup.find('div', {'class': 'nickname'}).text
        birthplace = soup.find('div', {'class': 'birthplace'}).text
        return {'Birthday': bday, 'Nickname': nickname, 'Birthplace': birthplace}
    except:
        return {'Birthday': None, 'Nickname': None, 'Birthplace': None}

In [12]:
#create a url with actor name for wikipedia
movies_df['URL'] = movies_df['lead_actor'].map(lambda x: 'https://en.wikipedia.org/wiki/' + x.replace(' ', '_'))

In [13]:
#scrape wikipedia
movies_df['WikiData'] = movies_df['URL'].map(to_scrape)

In [14]:
#add fields to dataframe
movies_df['Birthday'] = movies_df['WikiData'].map(lambda x : x.get('Birthday'))
movies_df['Full Name'] = movies_df['WikiData'].map(lambda x : x.get('Nickname'))
movies_df['Birthplace'] = movies_df['WikiData'].map(lambda x : x.get('Birthplace'))

In [15]:
movies_df

,title,lead_actor,URL,WikiData,Birthday,Full Name,Birthplace
0,Avatar,Sam Worthington,https://en.wikipedia.org/wiki/Sam_Worthington,"{'Birthday': '1976-08-02', 'Nickname': 'Samuel...",1976-08-02,Samuel Henry John Worthington,"Godalming, Surrey, England, UK"
1,Pirates of the Caribbean: At World's End,Johnny Depp,https://en.wikipedia.org/wiki/Johnny_Depp,"{'Birthday': '1963-06-09', 'Nickname': 'John C...",1963-06-09,John Christopher Depp II,"Owensboro, Kentucky, U.S."
2,Spectre,Daniel Craig,https://en.wikipedia.org/wiki/Daniel_Craig,"{'Birthday': '1968-03-02', 'Nickname': 'Daniel...",1968-03-02,Daniel Wroughton Craig,"Chester, Cheshire, England"
3,The Dark Knight Rises,Christian Bale,https://en.wikipedia.org/wiki/Christian_Bale,"{'Birthday': '1974-01-30', 'Nickname': 'Christ...",1974-01-30,Christian Charles Philip Bale,"Haverfordwest, Pembrokeshire, Wales"
4,John Carter,Taylor Kitsch,https://en.wikipedia.org/wiki/Taylor_Kitsch,"{'Birthday': None, 'Nickname': None, 'Birthpla...",None,None,None
5,Spider-Man 3,Tobey Maguire,https://en.wikipedia.org/wiki/Tobey_Maguire,"{'Birthday': '1975-06-27', 'Nickname': 'Tobias...",1975-06-27,Tobias Vincent Maguire,"Santa Monica, California, U.S."
6,Tangled,Zachary Levi,https://en.wikipedia.org/wiki/Zachary_Levi,"{'Birthday': '1980-09-29', 'Nickname': 'Zachar...",1980-09-29,Zachary Levi Pugh,"Lake Charles, Louisiana, U.S."
7,Avengers: Age of Ultron,Robert Downey Jr.,https://en.wikipedia.org/wiki/Robert_Downey_Jr.,"{'Birthday': '1965-04-04', 'Nickname': 'Robert...",1965-04-04,Robert John Downey Jr.,"New York City, New York, U.S."
8,Harry Potter and the Half-Blood Prince,Daniel Radcliffe,https://en.wikipedia.org/wiki/Daniel_Radcliffe,"{'Birthday': '1989-07-23', 'Nickname': 'Daniel...",1989-07-23,Daniel Jacob Radcliffe,"London, England"
9,Batman v Superman: Dawn of Justice,Ben Affleck,https://en.wikipedia.org/wiki/Ben_Affleck,"{'Birthday': '1972-08-15', 'Nickname': 'Benjam...",1972-08-15,Benjamin Géza Affleck-Boldt,"Berkeley, California, U.S."


In [16]:
#create sqlite engine
engine = create_engine("sqlite:///movieactors.sqlite")

In [17]:
#drop wikipedia info dictionary
movies_final = movies_df.drop(['WikiData'], axis=1)

In [18]:
#load movies_final dataframe to sqlite
import sqlite3

conn = sqlite3.connect('movieactors.sqlite')

movies_final.to_sql('tab', conn, if_exists='replace', index=False)
